In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

TEST_MODE = 0 # Testing macro

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

if TEST_MODE:
    for dirname, _, filenames in os.walk('/kaggle/input'):
        for filename in filenames:
            print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install --upgrade pip


from scipy.fft import fft, fftfreq
from scipy import signal
import numpy as np # linear algebraf
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import warnings
from joblib import Parallel, delayed, parallel_backend



pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
import seaborn as sns


from pandas.testing import assert_frame_equal

%run ./utility-functions.ipynb
%run ./SEED-CONSTANTS.ipynb

print(f"\x1b[32mSEED: {SEED}\x1b[0m")
# Global utitlity functions are in separate notebook
if TEST_MODE: print("setup complete")

utility_functions imports setup complete


In [5]:
!python --version

Python 3.9.10


In [25]:
def get_raw_windows(dfList_exp1, dfList_exp2, window_size, step_width, user_idx_set, scaler, num_sample_points_per_exp, EMA_per_win_span=None, SMA_per_win_winsize=None, 
                    Butter_per_win_argdict=None, verbose=1):

    # preparing data
    exp1_df_user_set_dict, exp2_df_user_set_dict = {user_key: dfList_exp1[user_key] for user_key in user_idx_set}, {user_key: dfList_exp2[user_key] for user_key in user_idx_set}
    print(f"len(exp1_df_user_set_dict): {len(exp1_df_user_set_dict)}")
    print(f"len(exp2_df_user_set_dict): {len(exp2_df_user_set_dict)}")
    
    # do not scale exp1 on its own, but instead use exp2 scaler from next steps to scale exp1
    exp1_data_prep_dict = MakeRawXExpDict(users_exp_df_dict=exp1_df_user_set_dict, window_size=window_size, step=step_width, numSamplePoints=num_sample_points_per_exp, 
                                          scaler=None, exp_num=1, EMA_per_win_span=EMA_per_win_span, SMA_per_win_winsize=SMA_per_win_winsize,
                                          Butter_per_win_argdict=Butter_per_win_argdict, verbose=verbose)
    
    
    exp2_data_prep_dict = MakeRawXExpDict(users_exp_df_dict=exp2_df_user_set_dict, window_size=window_size, step=step_width, numSamplePoints=num_sample_points_per_exp, 
                                          scaler=scaler, exp_num=2, EMA_per_win_span=EMA_per_win_span, SMA_per_win_winsize=SMA_per_win_winsize, 
                                          Butter_per_win_argdict=Butter_per_win_argdict, verbose=verbose)

    fitted_scaler_exp2_dict = exp2_data_prep_dict["fitted_scaler_exp_dic"]
    X_exp1_dict, X_exp2_dict = exp1_data_prep_dict["raw_X_exp_dict"], exp2_data_prep_dict["raw_X_exp_dict"]

    if verbose: print(f"Done getting raw wondows for window_size: {window_size}")
    return X_exp1_dict, X_exp2_dict, fitted_scaler_exp2_dict

    
def get_raw_windows_user_47(dfList_exp1_user_47, dfList_exp2_user_47, window_size, step_width, scaler, num_sample_points_per_exp, 
                            EMA_per_win_span=None, SMA_per_win_winsize=None, Butter_per_win_argdict=None, verbose=1):


    exp1_df_dict_user_47, exp2_df_dict_user_47 = {47: dfList_exp1_user_47[0]}, {47: dfList_exp2_user_47[0]}
    
    
    # Loading exp1 data:
    # 47) accel_count: 22777, gyro_count: 22226
    exp1_data_prep_dict_user_47 = MakeRawXExpDict(users_exp_df_dict=exp1_df_dict_user_47, window_size=window_size, step=step_width, 
                                                  numSamplePoints=num_sample_points_per_exp, scaler=None, exp_num=1, #+4000
                                                  EMA_per_win_span=EMA_per_win_span, SMA_per_win_winsize=SMA_per_win_winsize, 
                                                  Butter_per_win_argdict=Butter_per_win_argdict, verbose=verbose)
    
    # Loading exp2 data:
    # 47) accel_count: 17718, gyro_count: 18353
    exp2_data_prep_dict_user_47 = MakeRawXExpDict(users_exp_df_dict=exp2_df_dict_user_47, window_size=window_size, step=step_width, 
                                                  numSamplePoints=18000, scaler=scaler, exp_num=2, # has to be manually chosen
                                                  EMA_per_win_span=EMA_per_win_span, SMA_per_win_winsize=SMA_per_win_winsize, 
                                                  Butter_per_win_argdict=Butter_per_win_argdict, verbose=verbose)
    
    
    fitted_scaler_exp2_dict_user_47 = exp2_data_prep_dict_user_47["fitted_scaler_exp_dic"]
    X_exp1_dict_user_47, X_exp2_dict_user_47 = exp1_data_prep_dict_user_47["raw_X_exp_dict"], exp2_data_prep_dict_user_47["raw_X_exp_dict"]
    
    # ---- adjusting the len_exp2_user_47
    total_samples_per_user=len(X_exp1_dict_user_47[47])
    print(f"len(X_exp1_dict_user_47[47]): {len(X_exp1_dict_user_47[47])}")
    
    len_exp2_user_47 = len(X_exp2_dict_user_47[47])
    print(f"len_exp2_user_47: {len_exp2_user_47}")
    np.random.seed(SEED+len_exp2_user_47)
    X_exp2_dict_user_47[47] = X_exp2_dict_user_47[47] + [X_exp2_dict_user_47[47][idx] for idx in np.random.choice(len_exp2_user_47,
                                                                                                      size=total_samples_per_user-len_exp2_user_47,
                                                                                                      replace=False)]
    # ----
    
    return X_exp1_dict_user_47, X_exp2_dict_user_47, fitted_scaler_exp2_dict_user_47



def append_user_47_to_data(X_exp1_dict, X_exp2_dict, fitted_scaler_exp2_dict, all_user_set, X_exp1_dict_user_47, X_exp2_dict_user_47, fitted_scaler_exp2_dict_user_47, verbose=0):
    user_47_idx = len(all_user_set)
    
    if user_47_idx not in all_user_set:
        all_user_set.append(user_47_idx)
        
    user_47_key = user_47_idx
    X_exp1_dict[user_47_key] = X_exp1_dict_user_47[47]
    X_exp2_dict[user_47_key] = X_exp2_dict_user_47[47]
    fitted_scaler_exp2_dict[user_47_key] = fitted_scaler_exp2_dict_user_47[47]

    if verbose:
        print(f"user_47_idx: {user_47_idx}")
        print(f"np.unique(all_user_set): {np.unique(all_user_set)}")

        print(f"X_exp1_dict.keys(): {X_exp1_dict.keys()}")
        print(f"X_exp2_dict.keys(): {X_exp2_dict.keys()}")
        print(f"fitted_scaler_exp2_dict.keys(): {fitted_scaler_exp2_dict.keys()}")
        
    return X_exp1_dict, X_exp2_dict, fitted_scaler_exp2_dict

In [29]:
def apply_butter(df, filter_order=10, cut_off_freq=10, sampling_freq=100, plot=False, filtfilt=1):
    N = np.array(df).shape[0]
    # sample spacing
    T = 1.0 / sampling_freq

    sos = signal.butter(filter_order, cut_off_freq, 'lowpass', fs=sampling_freq, output='sos')
    # sig = np.array(df)
    sig = df.to_numpy()
    
    if filtfilt:
        # this one eliminates the delay 
        y = signal.sosfiltfilt(sos, sig) #filtered time domain signal
    else:
        y = signal.sosfilt(sos, sig) #filtered time domain signal


    yf = fft(y)
    xf = fftfreq(N, T)[:N//2]
    
    
    if plot:
        plt.plot(xf, 2.0/N * np.abs(yf[0:N//2]))
        plt.grid()
        plt.show()
    
    return y


def get_ffted_df(df, filter_order, cut_off_freq, sampling_freq, filtfilt):
    
    ffted_df = df.copy()
    ffted_df['x_a'] = apply_butter(ffted_df['x_a'], filter_order=filter_order, cut_off_freq=cut_off_freq, sampling_freq=sampling_freq, filtfilt=filtfilt).astype("float32")
    ffted_df['y_a'] = apply_butter(ffted_df['y_a'], filter_order=filter_order, cut_off_freq=cut_off_freq, sampling_freq=sampling_freq, filtfilt=filtfilt).astype("float32")
    ffted_df['z_a'] = apply_butter(ffted_df['z_a'], filter_order=filter_order, cut_off_freq=cut_off_freq, sampling_freq=sampling_freq, filtfilt=filtfilt).astype("float32")

    ffted_df['x_g'] = apply_butter(ffted_df['x_g'], filter_order=filter_order, cut_off_freq=cut_off_freq, sampling_freq=sampling_freq, filtfilt=filtfilt).astype("float32")
    ffted_df['y_g'] = apply_butter(ffted_df['y_g'], filter_order=filter_order, cut_off_freq=cut_off_freq, sampling_freq=sampling_freq, filtfilt=filtfilt).astype("float32")
    ffted_df['z_g'] = apply_butter(ffted_df['z_g'], filter_order=filter_order, cut_off_freq=cut_off_freq, sampling_freq=sampling_freq, filtfilt=filtfilt).astype("float32")
    
    return ffted_df

def get_EMAed_df(df, span):
    
    EMAed_df = df.copy()
    EMAed_df['x_a'] = EMAed_df['x_a'].ewm(span=span, adjust=False).mean().astype("float32")
    EMAed_df['y_a'] = EMAed_df['y_a'].ewm(span=span, adjust=False).mean().astype("float32")
    EMAed_df['z_a'] = EMAed_df['z_a'].ewm(span=span, adjust=False).mean().astype("float32")

    EMAed_df['x_g'] = EMAed_df['x_g'].ewm(span=span, adjust=False).mean().astype("float32")
    EMAed_df['y_g'] = EMAed_df['y_g'].ewm(span=span, adjust=False).mean().astype("float32")
    EMAed_df['z_g'] = EMAed_df['z_g'].ewm(span=span, adjust=False).mean().astype("float32")
    
    return EMAed_df


def get_SMAed_df(df, winsize):
    
    SMAed_df = df.copy()
    # min_periods: Minimum number of observations in window required to have a value, 
    # min_periods=0 since we do not want the result to be shorter in length, and there is non nan value
    SMAed_df['x_a'] = SMAed_df['x_a'].rolling(window=winsize, min_periods=0).mean()
    SMAed_df['y_a'] = SMAed_df['y_a'].rolling(window=winsize, min_periods=0).mean()
    SMAed_df['z_a'] = SMAed_df['z_a'].rolling(window=winsize, min_periods=0).mean()

    SMAed_df['x_g'] = SMAed_df['x_g'].rolling(window=winsize, min_periods=0).mean()
    SMAed_df['y_g'] = SMAed_df['y_g'].rolling(window=winsize, min_periods=0).mean()
    SMAed_df['z_g'] = SMAed_df['z_g'].rolling(window=winsize, min_periods=0).mean()
    
    return SMAed_df


def get_ffted_dfList(dfList_exp, filter_order=10, cut_off_freq=10, sampling_freq=100, filtfilt=1):
    
    ffted_dfList_exp = []
    for df in dfList_exp:
        
        ffted_df=get_ffted_df(df, filter_order=filter_order, cut_off_freq=cut_off_freq, sampling_freq=sampling_freq, filtfilt=filtfilt)
        
        ffted_dfList_exp.append(ffted_df)
        
    return ffted_dfList_exp

def get_EMAed_dfList(dfList_exp, span):
    
    EMAed_dfList_exp = []
    for df in dfList_exp:
        
        EMAed_df=get_EMAed_df(df, span)
        
        EMAed_dfList_exp.append(EMAed_df)
        
    return EMAed_dfList_exp

def get_SMAed_dfList(dfList_exp, winsize):
    
    SMAed_dfList_exp = []
    for df in dfList_exp:
        
        SMAed_df=get_SMAed_df(df, winsize)
        
        SMAed_dfList_exp.append(SMAed_df)
        
    return SMAed_dfList_exp

In [30]:
def matchAccelGyroData(accel, gyro):
    # Match the numbers by merge_asof to the higher length vector
    accel_count = accel.count().time_stamp
    gyro_count = gyro.count().time_stamp
    names =['sensor_id', 'time_stamp', 'x', 'y', 'z']
    if accel_count > gyro_count:
        df = pd.merge_asof(accel, gyro, on="time_stamp", direction='nearest')
        df = df.sort_values(by=['time_stamp'])
        df = df.dropna()
        accel = df[["sensor_id_x", "time_stamp", "x_x", "y_x", "z_x"]]
        gyro = df[["sensor_id_y", "time_stamp", "x_y", "y_y", "z_y"]]
    else:
        df = pd.merge_asof(gyro, accel, on="time_stamp", direction='nearest')
        df = df.sort_values(by=['time_stamp'])
        df = df.dropna()
        gyro = df[["sensor_id_x", "time_stamp", "x_x", "y_x", "z_x"]]
        accel = df[["sensor_id_y", "time_stamp", "x_y", "y_y", "z_y"]]

    accel.columns = names
    gyro.columns = names
    
    return {'accel': accel, 'gyro': gyro}

In [31]:
def getDataStats1(i, print_accel_gyro_array_size=1, print_na_df_array_size=0, begin_idx=500, end_idx=-500):
    
    #load the data
    names =['sensor_id', 'time_stamp', 'x', 'y', 'z']
#     if i!=8:
#         data = pd.read_csv('../input/wearable-assisted-ca/user{}_1.csv'.format(i), error_bad_lines = False, header=None, usecols = range(len(names)))
#     else:
    
    data = pd.read_csv(f'{os.getcwd()}/WACA_dataset/user{i}_1.csv', error_bad_lines = False, header=None, usecols = range(len(names)), dtype = str)
    data.columns = names
    data = data[(data.sensor_id == '10') | (data.sensor_id =='4')]
    data.head(10)


    types_dict = {'sensor_id': 'int32', 'time_stamp': 'float32', 'x': 'float32', 'y': 'float32', 'z': 'float32'}
    for col, col_type in types_dict.items():
        data[col] = data[col].astype(col_type)

    # find how many NAN values in the data
    data.isna().sum()

    # since only 7 NAN is a very small amount, drop them
    df = data.dropna()

    # get rid of begin and end noise
    #sort df and filter
    df = df.sort_values(by=['time_stamp'])
    df = df[begin_idx:end_idx]
    
    # cleaning extreme outliers
    df = df[(df.x < 5) & ( -5 < df.x) & (df.y < 5) & ( -5 < df.y) & (df.z < 5) & ( -5 < df.z) ]

    # Extract Accelerometer values and sort
    accel = df[df.sensor_id == 10]
    accel = accel.sort_values(by=['time_stamp'])

    # Extract gyro values and sort
    gyro = df[df.sensor_id == 4]
    gyro = gyro.sort_values(by=['time_stamp'])
    
    if print_accel_gyro_array_size:
        print("{}) accel_count: {}, gyro_count: {}".format(i, accel.count().time_stamp, gyro.count().time_stamp))
    
    result = matchAccelGyroData(accel, gyro)
    accel, gyro = result['accel'], result['gyro']
    
#     accel['x_a'] = accel['x'].ewm(span=40,adjust=False).mean()
#     accel['y_a'] = accel['y'].ewm(span=40,adjust=False).mean()
#     accel['z_a'] = accel['z'].ewm(span=40,adjust=False).mean()

#     gyro['x_g'] = gyro['x'].ewm(span=40,adjust=False).mean()
#     gyro['y_g'] = gyro['y'].ewm(span=40,adjust=False).mean()
#     gyro['z_g'] = gyro['z'].ewm(span=40,adjust=False).mean()
    
    accel['x_a'] = accel['x']
    accel['y_a'] = accel['y']
    accel['z_a'] = accel['z']

    gyro['x_g'] = gyro['x']
    gyro['y_g'] = gyro['y']
    gyro['z_g'] = gyro['z']
    
    left = accel[["time_stamp", "x_a", "y_a", "z_a"]]
    right = gyro[["time_stamp", "x_g", "y_g", "z_g"]].set_index('time_stamp')
    df = left.join(right, on='time_stamp')

    if print_na_df_array_size:
        print("{}) na_count: {}, df count: {}".format(i, df.isna().sum().sum(), df.count().time_stamp))
    
    return {"accel":accel.count().time_stamp, "gyro": gyro.count().time_stamp, "df": df, "userIdx": i}

In [32]:
def getDataStats2(i, print_accel_gyro_array_size=1, print_na_df_array_size=0, begin_idx=500, end_idx=-500):
    
    #load the data
    names =['sensor_id', 'time_stamp', 'x', 'y', 'z']
#     if i!=8:
#         data = pd.read_csv('../input/wearable-assisted-ca/user{}_1.csv'.format(i), error_bad_lines = False, header=None, usecols = range(len(names)))
#     else:
    
    data = pd.read_csv(f'{os.getcwd()}/WACA_dataset/user{i}_2.csv', error_bad_lines = False, header=None, usecols = range(len(names)), dtype = str)
    data.columns = names
    data = data[(data.sensor_id == '10') | (data.sensor_id =='4')]
    data.head(10)


    types_dict = {'sensor_id': 'int32', 'time_stamp': 'float32', 'x': 'float32', 'y': 'float32', 'z': 'float32'}
    for col, col_type in types_dict.items():
        data[col] = data[col].astype(col_type)

    # find how many NAN values in the data
    data.isna().sum()

    # since only 7 NAN is a very small amount, drop them
    df = data.dropna()

    # get rid of begin and end noise
    #sort df and filter
    df = df.sort_values(by=['time_stamp'])
    df = df[begin_idx:end_idx]
    
    # cleaning extreme outliers
    df = df[(df.x < 5) & ( -5 < df.x) & (df.y < 5) & ( -5 < df.y) & (df.z < 5) & ( -5 < df.z) ]

    # Extract Accelerometer values and sort
    accel = df[df.sensor_id == 10]
    accel = accel.sort_values(by=['time_stamp'])

    # Extract gyro values and sort
    gyro = df[df.sensor_id == 4]
    gyro = gyro.sort_values(by=['time_stamp'])
    
    if print_accel_gyro_array_size:
        print("{}) accel_count: {}, gyro_count: {}".format(i, accel.count().time_stamp, gyro.count().time_stamp))
    
    result = matchAccelGyroData(accel, gyro)
    accel, gyro = result['accel'], result['gyro']
    
#     accel['x_a'] = accel['x'].ewm(span=40,adjust=False).mean()
#     accel['y_a'] = accel['y'].ewm(span=40,adjust=False).mean()
#     accel['z_a'] = accel['z'].ewm(span=40,adjust=False).mean()

#     gyro['x_g'] = gyro['x'].ewm(span=40,adjust=False).mean()
#     gyro['y_g'] = gyro['y'].ewm(span=40,adjust=False).mean()
#     gyro['z_g'] = gyro['z'].ewm(span=40,adjust=False).mean()

    accel['x_a'] = accel['x']
    accel['y_a'] = accel['y']
    accel['z_a'] = accel['z']

    gyro['x_g'] = gyro['x']
    gyro['y_g'] = gyro['y']
    gyro['z_g'] = gyro['z']
    
    left = accel[["time_stamp", "x_a", "y_a", "z_a"]]
    right = gyro[["time_stamp", "x_g", "y_g", "z_g"]].set_index('time_stamp')
    df = left.join(right, on='time_stamp')

    if print_na_df_array_size:
        print("{}) na_count: {}, df count: {}".format(i, df.isna().sum().sum(), df.count().time_stamp))
    
    return {"accel":accel.count().time_stamp, "gyro": gyro.count().time_stamp, "df": df, "userIdx": i}

In [ ]:
# sample_rate = 10 #Hz
# #3352843.3
# x = np.array([318.45,302.78,316.47,334.14,333.41,326.15,320.07,318.68,314.12,308.64,
#               300.15,304.33,318.42,322.72,329.56,339.18,338.03,343.27,351.44,353.23,
#               352.35,352.88,353.43,352.14,351.28,352.82,353.36,353.35,353.19,353.82])

# mn=np.mean(x)
# print(f' mean = {mn:.3f} unit')
# print(f' sum x[i]**2  : {np.sum(x**2) :.1f} unit^2 ')


# print(f' n *sum X[k]**2   : {spectral_energy(x) :.1f} unit^2 ')

# Using SVM in a real-world Scenario with multiple genuine users and intruders
Source: https://datascience.stackexchange.com/questions/23623/what-is-the-best-way-to-classify-data-not-belonging-to-set-of-classes

Stage 1: 
    Use one-class SVM to assign those images that do not belong to the set of predefined classes as the 9-th class.

Stage 2:
    For those images that passes through your filter, let the multi-class SVM assign them to one of the 8 classes.

Loading data:

In [50]:
def load_data_frames(user_ids, begin_idx, end_idx, min_len):
    '''
    input: 
        user_ids: list of approved user_ids after exploratory data analysis
        begin_idx: the index before which data is discarded for user i
        end_idx: the index after which data is discarded for user i
        min_len: the minimum length that a dataframe has to be after cutting of both endings
        
    output:
        {dfList_exp1, dfList_exp2}: return dfList for exp1 and exp2 of the selected user_ids
    '''
    print("Loading exp1 data:")
    dfList_exp1 = []
    for i in user_ids:
        dic = getDataStats1(i, begin_idx=begin_idx, end_idx=end_idx)

        if(dic['accel']<min_len):
            raise Exception("The Stream is shorter than {}".format(min_len))

        dfList_exp1 = dfList_exp1 + [dic['df'].reset_index(drop=True)]


    print("Loading exp2 data:")
    dfList_exp2 = []
    for i in user_ids:
        dic = getDataStats2(i, begin_idx=begin_idx, end_idx=end_idx)

        if(dic['accel']<min_len):
            raise Exception("The Stream is shorter than {}".format(min_len))

        dfList_exp2 = dfList_exp2 + [dic['df'].reset_index(drop=True)]
    #     dfList = dfList + [dic['df']]
    
    return {"dfList_exp1": dfList_exp1, "dfList_exp2": dfList_exp2}

In [52]:
def getRawDataChunks(df, windows, scaler=None, user_key=None, exp_num=None, EMA_per_win_span=None, SMA_per_win_winsize=None, Butter_per_win_argdict=None, verbose=0):
    a = []
    df = df.copy().drop(columns=["time_stamp"])
    # df_array = df.to_numpy()
    
    if EMA_per_win_span!=None and SMA_per_win_winsize!=None:
        raise Exception("cannot apply both EMA_per_win_span and SMA_per_win_winsize at the same time.")
        
    if scaler!=None:
        # print(df_array.shape)
        scaler = get_new_scaler_dict[scaler]
        # scaler = scaler().fit(df_array)
        scaler = scaler().fit(df)
        # df_array = scaler.transform(df_array)
        df_array = scaler.transform(df)
        # scaled_df = pd.DataFrame(data=df_array, columns = df.columns, dtype=df_array.dtype)
        scaled_df = pd.DataFrame(data=df_array, columns = df.columns, dtype=df_array.dtype)
        df = scaled_df
        
    if verbose: print(f"getRawDataChunks - user_key: {user_key}, exp_num: {exp_num}, scaler: {scaler}")
    
    verbose_c = verbose
    for i in range(len(windows)):
        # a.append(df_array[windows[i], :]) #CNN
        
        df_window=df.iloc[windows[i], :].copy()

        # per window is currently only available for exp 1 
        if scaler==None:
            
            if Butter_per_win_argdict!=None:
                
                
                filter_order = Butter_per_win_argdict["filter_order"]
                cut_off_freq = Butter_per_win_argdict["cut_off_freq"]
                sampling_freq = Butter_per_win_argdict["sampling_freq"]
                filtfilt = Butter_per_win_argdict["filtfilt"]
                
                # if verbose_c==1: df_window["x_a"][0:500].plot()
                    
                df_window = get_ffted_df(df_window, filter_order=filter_order, cut_off_freq=cut_off_freq, sampling_freq=sampling_freq, filtfilt=filtfilt)

                # if verbose_c: 
                #     df_window["x_a"][0:500].plot()
                #     print(f"Butter_per_win_argdict: {Butter_per_win_argdict}")
                    
            # exponential moving average
            if EMA_per_win_span!=None:
                
                df_window = get_EMAed_df(df_window, span=EMA_per_win_span)
                if verbose_c==1: df_window["x_a"][0:500].plot()
            # simple moving average
            elif SMA_per_win_winsize!=None:

                df_window = get_SMAed_df(df_window, winsize=SMA_per_win_winsize)
                if verbose_c==1: df_window["x_a"][0:500].plot()


        verbose_c=0
        a.append(df_window) #waca
    
#     print(len(a))
#     print(len(a))
#     return np.array([a])
    # return np.array(a), scaler #CNN
    
    return a, scaler #waca


def MakeRawXExpDict(users_exp_df_dict, window_size, step, numSamplePoints, scaler, exp_num, EMA_per_win_span=None, SMA_per_win_winsize=None, Butter_per_win_argdict=None,
                    verbose=1):
    '''
    return 
    X_exp_dic
    '''
    
    windows = getIndices(sampleSize=window_size, step=step, numSamplePoints=numSamplePoints)

    X_exp_dic = {}
    fitted_scaler_exp_dic={}
    for user_key in users_exp_df_dict:
        
        encoding_array_exp, fitted_scaler_exp = getRawDataChunks(users_exp_df_dict[user_key], windows, scaler=scaler, user_key=user_key, exp_num=exp_num,
                                                                 EMA_per_win_span=EMA_per_win_span, SMA_per_win_winsize=SMA_per_win_winsize, 
                                                                 Butter_per_win_argdict=Butter_per_win_argdict, verbose=verbose)
        
        X_exp_dic[user_key] = encoding_array_exp
        
        fitted_scaler_exp_dic[user_key]=fitted_scaler_exp
        
    return {"raw_X_exp_dict": X_exp_dic, "fitted_scaler_exp_dic": fitted_scaler_exp_dic}




def extract_features_and_scale_for_parallel_call_Owner(X_exp_reg_df_dict, scaler_clip, scaler_type, feature_extractor, feature_extractor_transformer, owner, verbose=0):
    
    X_exp_dic = {}
    
    X_exp_dic[owner] = {"profile_windows": feature_extractor(X_exp_reg_df_dict[owner]), "unknown_users_dict": {}, "fitted_feature_scaler": None}
        
    # do minmax scaling here
    if verbose: print(f"fit_transform {feature_extractor.__name__} features using scaler of type: {scaler_type}, for profile_windows of owner: {owner}")
    if verbose: print(f"owner: {owner}, exp_num: {2}, scaler: {scaler_type}, scaler_clip: {scaler_clip}")

    scaler = get_new_scaler_dict[scaler_type]
    scaler = scaler(clip=scaler_clip).fit(X_exp_dic[owner]["profile_windows"])
    X_exp_dic[owner]["fitted_feature_scaler"] = scaler

    X_exp_dic[owner]["profile_windows"] = feature_extractor_transformer(X_exp_dic[owner]["profile_windows"], X_exp_dic[owner]["fitted_feature_scaler"])
        
        
    return {"owner": owner, "X_exp_dic": X_exp_dic}


def MakeXExpDicOwner(X_exp_reg_df_dict, scaler_clip=True, scaler_type="MinMaxScaler", feature_extractor=None, feature_extractor_transformer=None, verbose=0):
    '''k
    ???
    return 
    X_exp_dic
    dfLists are of the same size.
    '''

    X_exp_dic = {}
    
#     for owner in X_exp_reg_df_dict.keys():

#         if verbose: print(f"raw exp2 samples of owner: {owner} are scaled already so we {feature_extractor.__name__}")
        
#         X_exp_dic[owner] = {"profile_windows": feature_extractor(X_exp_reg_df_dict[owner]), "unknown_users_dict": {}, "fitted_feature_scaler": None}
        
# #         #------- DEBUG--------
# #         if owner == 2: 
# #             X_exp_dic['debug']= X_exp_dic[owner], X_exp_reg_df_dict[owner]
# #             return X_exp_dic
        
# #         #------- DEBUG--------
        
#         # do minmax scaling here
#         if verbose: print(f"fit_transform {feature_extractor.__name__} features using scaler of type: {scaler_type}, for profile_windows of owner: {owner}")
#         if verbose: print(f"owner: {owner}, exp_num: {2}, scaler: {scaler_type}, scaler_clip: {scaler_clip}")
        
#         scaler = get_new_scaler_dict[scaler_type]
#         scaler = scaler(clip=scaler_clip).fit(X_exp_dic[owner]["profile_windows"])
#         X_exp_dic[owner]["fitted_feature_scaler"] = scaler

#         X_exp_dic[owner]["profile_windows"] = feature_extractor_transformer(X_exp_dic[owner]["profile_windows"], X_exp_dic[owner]["fitted_feature_scaler"])
        
        
        
    with parallel_backend('loky', n_jobs=-1):
        results = Parallel(n_jobs=-1, verbose=100)(delayed(extract_features_and_scale_for_parallel_call_Owner)
                                                   (X_exp_reg_df_dict, scaler_clip, scaler_type, feature_extractor, feature_extractor_transformer, owner, verbose=0) 
                                                   for owner in X_exp_reg_df_dict.keys())
    
    for item in results:
        owner = item["owner"]
        X_exp_dic[owner] = item["X_exp_dic"][owner]
        
    
    return X_exp_dic
    


def extract_features_and_scale_for_parallel_call_Unknown(X_exp_unknown_df_dict, X_exp_dic, fitted_raw_scaler_dict, feature_extractor, feature_extractor_transformer, owner, verbose=0):
    if fitted_raw_scaler_dict:
        
        if verbose: print(f"scaling exp1 unknown windows for when user: {owner}, is owner")
        
        for unknown_user in X_exp_unknown_df_dict.keys():
            # scale the unkown window using the scaler fitted on raw registeration data and then extract the unknown features
            X_exp_dic[owner]["unknown_users_dict"][unknown_user] = feature_extractor(transform_user_windows(X_exp_unknown_df_dict[unknown_user], fitted_raw_scaler_dict[owner]))
            
            # get the scaler fitted on the registeration features
            fitted_feature_scaler = X_exp_dic[owner]["fitted_feature_scaler"]
            
            # scale the unkown features using the registeration feature scaler
            X_exp_dic[owner]["unknown_users_dict"][unknown_user] = feature_extractor_transformer(X_exp_dic[owner]["unknown_users_dict"][unknown_user], fitted_feature_scaler)
            
        if verbose: print(f"done scaling exp1 unknown windows for when user: {owner}, is owner")
        
    return {"owner": owner, "unknown_users_dict": X_exp_dic[owner]["unknown_users_dict"]}
    
def MakeXExpDicUnknown(X_exp_unknown_df_dict, X_exp_dic, fitted_raw_scaler_dict, feature_extractor, feature_extractor_transformer):
    '''k
    ???
    return 
    X_exp_dic
    dfLists are of the same size.
    '''

#     for owner in X_exp_unknown_df_dict.keys():
#         if fitted_raw_scaler_dict:
#             print(f"scaling exp1 unknown windows for when user: {owner}, is owner")
#             for unknown_user in X_exp_unknown_df_dict.keys():
#                 # scale the unkown window using the scaler fitted on raw registeration data and then extract the unknown features
#                 X_exp_dic[owner]["unknown_users_dict"][unknown_user] = feature_extractor(transform_user_windows(X_exp_unknown_df_dict[unknown_user], fitted_raw_scaler_dict[owner]))
                
#                 # get the scaler fitted on the registeration features
#                 fitted_feature_scaler = X_exp_dic[owner]["fitted_feature_scaler"]
                
#                 # scale the unkown features using the registeration feature scaler
#                 X_exp_dic[owner]["unknown_users_dict"][unknown_user] = feature_extractor_transformer(X_exp_dic[owner]["unknown_users_dict"][unknown_user], fitted_feature_scaler)
            
            
    with parallel_backend('loky', n_jobs=-1):
        results = Parallel(n_jobs=-1, verbose=100)(delayed(extract_features_and_scale_for_parallel_call_Unknown)(X_exp_unknown_df_dict, X_exp_dic, fitted_raw_scaler_dict, 
                                                                                                                         feature_extractor, feature_extractor_transformer,
                                                                                                                         owner) for owner in X_exp_unknown_df_dict.keys())

    
    for item in results:
        owner = item["owner"]
        unknown_users_dict = item["unknown_users_dict"]
        X_exp_dic[owner]["unknown_users_dict"] = unknown_users_dict
    
    
    return X_exp_dic



def transform_user_windows(X_exp, fitted_scaler):
    
    
    transformed_X_exp = []
    
#     print(X_exp[0].shape)
    for window in X_exp:
        scaled_array = fitted_scaler.transform(window)
        scaled_window_df = pd.DataFrame(data=scaled_array, columns = window.columns, dtype=scaled_array.dtype)
        transformed_X_exp.append(scaled_window_df)
        
    return transformed_X_exp

In [4]:
print(20*'-' + "\x1b[32mPreprocessing utility functions imported\x1b[0m" + 20*'-')

--------------------Preprocessing utility functions imported--------------------
